In [ ]:
import Pkg; Pkg.activate(".")
using Revise, Backboner

In [ ]:
backbone = readpdb("test/data/1ASS.pdb")[1].backbone

In [ ]:
import BioStructures: Transformation
import Rotations: QuatRotation
import Statistics: mean

In [ ]:
residuerotation(coords::AbstractMatrix) = QuatRotation(Transformation(Backboner.Protein.STANDARD_TRIANGLE_ANGSTROM, coords).rot)

In [ ]:
# Algorithm 21 (x1: N, x2: Ca, x3: C)
function rigid_from_3points(x1::AbstractVector, x2::AbstractVector, x3::AbstractVector)
    v1 = x3 - x2
    v2 = x1 - x2
    e1 = normalize(v1)
    u2 = v2 - e1 * (e1'v2)
    e2 = normalize(u2)
    e3 = cross(e1, e2)
    R = [e1 e2 e3]
    t = x2
    return R, t
end

In [ ]:
QuatRotation.(eachslice(Backboner.backbone_to_locs_and_rots(backbone)[2], dims=3))

In [ ]:
new_bb = Backboner.locs_and_rots_to_backbone(
    hcat([mean(backbone.coords[:, i:i+2], dims=2) for i in 1:3:length(backbone)]...),
    stack(residuerotation.([backbone.coords[:, i:i+2] for i in 1:3:length(backbone)]))
)

In [ ]:
Backboner.backbone_to_locs_and_rots(backbone)[2]